Práctica realizada por: Armando Torner Marchesi

# Examen ETL: SPARK 02/02

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

Los RDD son la abstracción que Spark define para manejar conjuntos de datos. Un RDD es una colección de elementos sobre los que se puede acceder en paralelo desde los diferentes nodos de nuestro clúster Spark. Una de sus características más atractivas es que es tolerante a fallos, es decir, si el nodo que está procesando una parte del RDD falla, o está yendo demasiado lento, el proceso se relanzará en otro nodo en mejor estado para evitar el fallo o la ralentización del proceso global.

En un RDD de gran tamaño, recalcular dos veces una misma transformación conllevará un sobrecoste de tiempo de ejecución. Para evitarlo, Spark provee dos métodos, persist() y cache(), para guardar transformaciones intermedias y poder reutilizarlas, evitando repetir varias veces un mismo grupo de cálculos. La diferencia entre ambos es que cache() guarda los RDD en el nivel de almacenamiento por defecto (como objetos de Java serializados en la memoria principal) y persist() permite, mediante un parámetro opcional, elegir un nivel de almacenamiento.

Por su parte, collect() retorna todos los elementos de un RDD como una lista de Python. Este método es útil cuando se obtiene un RDD suficientemente pequeño como para que pueda ser procesado por el programa controlador. En pocas palabras, collect() trae de vuelta los datos paralelizados para Spark a un formato que puede ser manejado por el lenguaje del programa controlador (en este caso, una lista Python)

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

Una carga de datos consta principalmente de tres pasos. El primero sería cargar los paquetes necesarios para poder llevar a cabo la carga del fichero de datos. En segundo lugar, es recomendable establecer una variable con la ruta específica donde se encuentra el archivo. Esto es muy importante de cara a que si varía la ruta del archivo, no sea necesario cambiar la ruta en varios sitios del documento, sino hacerlo en uno sólo y que se replique en todo el documento al modifical el valor de la variable. Y por último, la carga del archivo de datos 'per se', atendiendo a distintos criterios (tipo de archivo, separador, filtrado de alguna columna o cabecera, etc ...) y por tanto utilizando distintas funciones según lo necesario para cada caso particular.

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

Un problema que puede empeorar sustancialmente los datos es la falta de registros en variables importantes de los mismos. Esto, de cara a cualquier tipo de análisis es un gran problema. La solución puede ir por dos caminos muy diferentes. El primero es la eliminación de esos datos (ya sea por la eliminación de variables o de los registros afectados), pero podemos tener una gran pérdida de información. La otra solución es estimar esos datos de alguna forma (media, valores similares, etc), pero en este caso se corre el riesgo de que la información estimada sea erronea. 

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

In [1]:
sc.stop()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local","Simple App")

In [3]:
data_file = "./partidosLigaNBA.csv"
partidos = sc.textFile(data_file)

Hay que hacer un split, es decir, separar las variables y eliminar el header

In [4]:
# Primero quitamos el header para pasarle el output correcto a la funcion creada
header = partidos.first()
partidos = partidos.filter(lambda x : x != header)

Vemos que hay un pequeño error con las fechas, vamos a parsearlas creando una función, para después poder usar el año de la fecha para distintos cálculos

In [5]:
#Cargamos librerías
import datetime
import time

#Definimos la función

def formatoFecha(elemento):

    elemento=time.strptime(elemento, "%a, %b %d, %Y")

    return datetime.datetime(elemento[0],elemento[1],elemento[2],elemento[3])

Ahora creamos una función para parsear el RDD

In [8]:
# Cargamos librerías necesarias:
from pyspark.sql import Row
import datetime

#  Introducimos a la función un RDD sin Header
def parseadoNBA(lineas):
    
    #Split por ':'
    elementos = lineas.split(":")  
    
    #El split separa la hora y los minutos, asi que los juntamos en el elemento 1 (usamos para ello "+")
    elementos[1]= elementos[1]+":"+elementos[2]  
    
    #Como hemos incorporado el elemento 2 al 1, podemos eliminarlo
    del elementos[2]
    
    #Devolvemos la línea con el formato correcto
    return(Row(
        Date=formatoFecha(elementos[0]), #Metemos aqui la función que hemos creado antes
        Dstart = elementos[1],  
        Visitor= elementos[2],
        PTSvis  = int(elementos[3]),
        Home    = elementos[4],
        PTShom  = int(elementos[5])
    ))

In [7]:
# Aplicamos el parseado y vemos que todo está correcto
partidos_sep = partidos.map(parseadoNBA)
partidos_sep.take(3)

[Row(Date=datetime.datetime(2007, 10, 30, 0, 0), Dstart='"7:30 pm"', Home='Golden State Warriors', PTShom=96, PTSvis=117, Visitor='Utah Jazz'),
 Row(Date=datetime.datetime(2007, 10, 30, 0, 0), Dstart='"7:30 pm"', Home='Los Angeles Lakers', PTShom=93, PTSvis=95, Visitor='Houston Rockets'),
 Row(Date=datetime.datetime(2007, 10, 30, 0, 0), Dstart='"7:00 pm"', Home='San Antonio Spurs', PTShom=106, PTSvis=97, Visitor='Portland Trail Blazers')]

## Quinta tarea: Media de la diferencia de puntos por año

In [9]:
#Lo primero que hacemos es mapear poniedo como clave el año, y como valor la diferencia de puntos y un contador, por si el 
#numero de partidos por temporada no es el mismo:
anos_diferencias = partidos_sep.map(lambda x: (x[0].year, (abs(x[3]-x[4]),1)))
anos_diferencias.take(3)

[(2007, (21, 1)), (2007, (2, 1)), (2007, (9, 1))]

In [ ]:
# Con lo anterior agrupamos sumando la diferencia y el numero de partidos por año
anos_agrupado = anos_diferencias.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
anos_agrupado.take(3)

In [ ]:
# Ahora mapeamos para quedarnos con los años y realizar la media de las diferencias
anos_agrupado.map(lambda x: (x[0], x[1][0]/x[1][1])).collect()

## Sexta tarea: ¿Han jugado todos los equipos el mismo número de partidos? ¿ Si es que no a qué puede deberse?

Con un poco de conocimiento de la NBA, esta pregunta puede contestarse teóricamente. No han jugado todos los equipos el mismo número de partidos ya que cada temporada de la NBA tiene dos fases. La primera es la liga regular, en la que los equipos juegan los mismos partidos, y después, los mejores equipos de cada conferencia se clasifican para los Play-Off´s, que son eliminatorias que sólo juegan los equipos que han conseguido clasificarse. Por tanto, los equipos que más partidos juegan son los que se clasifican para la gran final, y los que menos, aquellos que no logran clasificarse para los Play-Off´s

In [ ]:
# Mapeamos para fijar como clave el equipo y la temporada.
partidos_equipo_temporada = partidos_sep.map(lambda x: ((x[0].year,x[2]), 1)).reduceByKey(lambda x,y: x+y)
partidos_equipo_temporada.take(15)

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Cleveland?

In [ ]:
# Filtramos para obtener todos los partidos de los Cavaliers:
partidos_CC = partidos_sep.filter(lambda x : x[2] == "Cleveland Cavaliers" or x[5] == "Cleveland Cavaliers")
# Una vez tenemos los partidos de Cleveland, filtramos para obtener los que se jugaron en el mes 1 (que es Enero):
partidos_CC_enero = partidos_CC.filter(lambda x: x[0].month == 1)
partidos_CC_enero.take(5)

In [ ]:
# Definimos una función que nos cuente las victorias independientemente de que sean locales o visitantes, para luego mapear el
# dataset filtrado:
def cuenta_victorias(x):
    victorias=0
    if x[2]=="Cleveland Cavaliers":
        if x[3]>x[4]:
            victorias=victorias+1
    if x[5]=="Cleveland Cavaliers":
        if x[3]<x[4]:
            victorias=victorias+1
    return victorias

In [ ]:
# Mapeamos y sumamos las victorias
partidos_CC_enero_sum = partidos_CC_enero.map(lambda x : cuenta_victorias(x)).sum()
partidos_CC_enero_sum

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

Entiendo que para esta comparativa vamos a contrastar las victorias locales con las victorias fuera de casa

In [12]:
# Filtramos los partidos de Golden State Warriors
partidos_warriors = partidos_sep.filter(lambda x : x[2] == "Golden State Warriors" or x[5] == "Golden State Warriors")

In [ ]:
# Modificamos la función que hemos creado antes para adaptarla a este caso concreto
def cuenta_victorias_localovisitante(x):
    victorias_local=0
    victorias_visitante=0
    if x[2]=="Golden State Warriors":
        if x[3]>x[4]:
            victorias_local=victorias_local+1
    if x[5]=="Golden State Warriors":
        if x[3]<x[4]:
            victorias_visitante=victorias_visitante+1
    return (victorias_local, victorias_visitante)

In [ ]:
# Sacamos la victoria o derrota para cada partido
partidos_warriors_ganados = partidos_warriors.map(lambda x : cuenta_victorias_localovisitante(x))

In [ ]:
# Reducimos y sumamos. Las victorias locales son la cifra de la izquierda, y las visitantes las de la derecha
partidos_warriors_ganados_total = partidos_warriors_ganados.reduce(lambda x,y: (x[0]+y[0],x[1]+y[1]))
partidos_warriors_ganados_total

## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com